In [2]:
import os
import json
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

autocast_questions = json.load(open('autocast_questions.json')) # from the Autocast dataset
test_questions = json.load(open('autocast_competition_test_set.json'))
test_ids = [q['id'] for q in test_questions]


In [3]:
# Filter the questions and labels
filtered_questions = [q for q in autocast_questions if q['id'] not in test_ids and q['answer'] is not None]
questions = [item['question'] for item in filtered_questions]
labels = [item['choices'][0] for item in filtered_questions]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_questions = tokenizer(questions, padding=True, truncation=True, max_length=128, return_tensors='tf')

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
input_ids_train, input_ids_test, attention_mask_train, attention_mask_test, y_train, y_test = train_test_split(
    encoded_questions['input_ids'], encoded_questions['attention_mask'], encoded_labels, test_size=0.2, random_state=42
)


KeyError: 0

In [2]:
# autocast_questions = json.load(open('autocast_questions.json')) # from the Autocast dataset
# test_questions = json.load(open('autocast_competition_test_set.json'))
test_ids = [q['id'] for q in test_questions]

## Create baseline models outputting random answers

In [3]:
def random_baseline_model(question):
    if question['qtype'] == 't/f':
        return np.random.random(size=2)
    elif question['qtype'] == 'mc':
        probs = np.random.random(size=len(question['choices']))
        return probs / probs.sum()
    elif question['qtype'] == 'num':
        return np.random.random()


def calibrated_random_baseline_model(question):
    if question['qtype'] == 't/f':
        pred_idx = np.argmax(np.random.random(size=2))
        pred = np.ones(2)
        pred[pred_idx] += 1e-5
        return pred / pred.sum()
    elif question['qtype'] == 'mc':
        pred_idx = np.argmax(np.random.random(size=len(question['choices'])))
        pred = np.ones(len(question['choices']))
        pred[pred_idx] += 1e-5
        return pred / pred.sum()
    elif question['qtype'] == 'num':
        return 0.5

## Get performance on the Autocast train set

Note that the Autocast dataset contains questions in the competition test set. Those should not be used.

In [4]:
def brier_score(probabilities, answer_probabilities):
    return ((probabilities - answer_probabilities) ** 2).sum() / 2

In [5]:
preds = []
answers = []
qtypes = []
# for question in autocast_questions:
#     if question['id'] in test_ids: # skipping questions in the competition test set
#         continue
#     if question['answer'] is None: # skipping questions without answer
#         continue
#     preds.append(calibrated_random_baseline_model(question))
#     if question['qtype'] == 't/f':
#         ans_idx = 0 if question['answer'] == 'no' else 1
#         ans = np.zeros(len(question['choices']))
#         ans[ans_idx] = 1
#         qtypes.append('t/f')
#     elif question['qtype'] == 'mc':
#         ans_idx = ord(question['answer']) - ord('A')
#         ans = np.zeros(len(question['choices']))
#         ans[ans_idx] = 1
#         qtypes.append('mc')
#     elif question['qtype'] == 'num':
#         ans = float(question['answer'])
#         qtypes.append('num')
#     answers.append(ans)

## Evaluate the model

In [6]:
tf_results, mc_results, num_results = [],[],[]
for p, a, qtype in zip(preds, answers, qtypes):
    if qtype == 't/f':
        tf_results.append(brier_score(p, a))
    elif qtype == 'mc':
        mc_results.append(brier_score(p, a))
    else:
        num_results.append(np.abs(p - a))

print(f"T/F: {np.mean(tf_results)*100:.2f}, MCQ: {np.mean(mc_results)*100:.2f}, NUM: {np.mean(num_results)*100:.2f}")
print(f"Combined Metric: {(np.mean(tf_results) + np.mean(mc_results) + np.mean(num_results))*100:.2f}")

T/F: nan, MCQ: nan, NUM: nan
Combined Metric: nan


/Users/abhishektiwari/Desktop/sem2/college_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/abhishektiwari/Desktop/sem2/college_env/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## Make predictions on test set

In [7]:
preds = []
for question in test_questions:
    preds.append(calibrated_random_baseline_model(question))

In [2]:
# if not os.path.exists('submission'):
#     os.makedirs('submission')

# with open(os.path.join('submission', 'predictions.pkl'), 'wb') as f:
#     pickle.dump(preds, f, protocol=2)

!cd submission && zip ../submission.zip ./* && cd ..

updating: predictions.pkl (deflated 55%)


In [2]:
!ls

BERTmodel.py                       example_submission.ipynb
README.md                          submission
autocast_competition_test_set.json submission.zip
autocast_questions.json
